## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Angoche,MZ,-16.2325,39.9086,78.51,81,94,6.20,overcast clouds
1,1,Broome,US,42.2506,-75.8330,35.62,77,90,3.00,light rain
2,2,Bambous Virieux,MU,-20.3428,57.7575,74.21,73,23,10.80,few clouds
3,3,Chokurdakh,RU,70.6333,147.9167,-15.12,99,100,6.31,overcast clouds
4,4,Junin,AR,-34.5838,-60.9433,62.96,56,100,8.79,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Angoche,MZ,-16.2325,39.9086,78.51,81,94,6.20,overcast clouds
5,5,Port Moresby,PG,-9.4431,147.1797,85.24,68,94,6.15,overcast clouds
17,17,Kapaa,US,22.0752,-159.3190,80.92,79,20,3.00,moderate rain
20,20,Wawa,NG,9.9022,4.4192,75.38,29,5,3.27,clear sky
23,23,Mana,GF,5.6592,-53.7767,78.46,88,100,8.41,overcast clouds
30,30,Buchanan,LR,5.8808,-10.0467,76.96,85,59,2.68,light rain
32,32,Dubbo,AU,-32.2500,148.6167,77.13,61,75,5.75,broken clouds
34,34,Hervey Bay,AU,-25.2986,152.8535,77.11,78,49,6.58,light rain
38,38,Cayenne,GF,4.9333,-52.3333,76.08,91,75,0.00,heavy intensity rain
43,43,Bubaque,GW,11.2833,-15.8333,81.28,70,5,10.16,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Angoche,MZ,78.51,overcast clouds,-16.2325,39.9086,
5,Port Moresby,PG,85.24,overcast clouds,-9.4431,147.1797,
17,Kapaa,US,80.92,moderate rain,22.0752,-159.3190,
20,Wawa,NG,75.38,clear sky,9.9022,4.4192,
23,Mana,GF,78.46,overcast clouds,5.6592,-53.7767,
30,Buchanan,LR,76.96,light rain,5.8808,-10.0467,
32,Dubbo,AU,77.13,broken clouds,-32.2500,148.6167,
34,Hervey Bay,AU,77.11,light rain,-25.2986,152.8535,
38,Cayenne,GF,76.08,heavy intensity rain,4.9333,-52.3333,
43,Bubaque,GW,81.28,clear sky,11.2833,-15.8333,


In [9]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'].astype(bool)]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Angoche,MZ,78.51,overcast clouds,-16.2325,39.9086,Guest House HESADA
5,Port Moresby,PG,85.24,overcast clouds,-9.4431,147.1797,Lamana Hotel (Port Moresby)
17,Kapaa,US,80.92,moderate rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
23,Mana,GF,78.46,overcast clouds,5.6592,-53.7767,LE SAMANA
30,Buchanan,LR,76.96,light rain,5.8808,-10.0467,Kamaneahn Hotel Inc
32,Dubbo,AU,77.13,broken clouds,-32.2500,148.6167,Comfort Inn Blue Lagoon
34,Hervey Bay,AU,77.11,light rain,-25.2986,152.8535,Shelly Bay Resort
38,Cayenne,GF,76.08,heavy intensity rain,4.9333,-52.3333,Hôtel Le Dronmi
43,Bubaque,GW,81.28,clear sky,11.2833,-15.8333,"Casa Dora, Bubaque"
45,Rikitea,PF,76.42,clear sky,-23.1203,-134.9692,People ThankYou


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.7)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))